In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as linalg

def calibrateCamera_Tsai(p, P):
    # p : homogeneous coordinates of pixel in the image frame
    # P : homogeneous coordinates of points in the world frame
    assert p.shape[0] == 3, "p : homogeneous coordinates of pixel in the image frame of 3 by n"
    assert P.shape[0] == 4, "P : homogeneous coordinates of points in the world frame"
    assert p.shape[1] == P.shape[1], "number of columns of p shold match with P"

    n = p.shape[1]
    p_uv = p[0:2,:]/p[2,:]

    Q = np.empty((0, 12))
    for i in range(n):
        Qi_0 = np.array([ [1,0, -p_uv[0,i]], [0, 1, -p_uv[1,i]]] )
        Qi = np.kron(Qi_0, P[:,i].T)
        Q = np.append(Q, Qi, axis=0)

    # 1. Find M_tilde using SVD

    U, S, VT = linalg.svd(Q)
    M_tilde = VT.T[:,-1].reshape(3,4)
    # print(M_tilde /M_cv) # M is determined up to scale

    # 2. RQ factorization to find K_tilde and R

    K_tilde, R = linalg.rq(M_tilde [:,0:3])

    # 3. Resolve the ambiguity of RQ factorization
    D = np.diag( np.sign(np.diag(K_tilde)) )
    K_tilde  = K_tilde@D
    R = D@R

    # 4. Find T
    T = linalg.solve(K_tilde, M_tilde[:,-1]).reshape(3,1)

    # 5. Recover scale

    s = 1/K_tilde[2,2]
    K = s*K_tilde
    M = s*M_tilde

    # 6. Resolve sign ambiguity
    if linalg.det(R) < 0:
        R = -R
        T = -T
        M = -M


    return K, R, T, M

In [2]:
Data = pd.read_csv("Marks2.csv")

u, v = Data["u"].values, Data["v"].values
X, Y, Z = Data["X"].values, Data["Y"].values, Data["Z"].values

In [ ]:
from matplotlib import cm

cmap = cm.get_cmap("Dark2", 51)

zmax = Z.max()
xmax = X.max()

fig, ax = plt.subplots(dpi=200)
ax.scatter(u, v, s=1, marker='x', color=cmap(Z/zmax))

In [4]:
def Pinhole6292(u, v, Xw, Yw, Zw):
    uv_homogenous = np.ones((3, len(u)))
    xyz_homogenous = np.ones((4, len(Xw)))
    
    uv_homogenous[0, :] = u
    uv_homogenous[1, :] = v
    
    xyz_homogenous[0, :] = Xw
    xyz_homogenous[1, :] = Yw
    xyz_homogenous[2, :] = Zw
    
    return calibrateCamera_Tsai(uv_homogenous, xyz_homogenous)

In [ ]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

File = "Marks2.csv"
Data = pd.read_csv(File)

# Data = Data[(1 < Data["Z"]) & (Data["Z"] < 11)]

# Data.sort_values(by="u", axis=0, inplace=True)

u, v = Data["u"].values, Data["v"].values
u -= u.mean()
v -= v.mean()
# u, v = Data["Xcam2"].values, Data["Ycam2"].values
X, Y, Z = Data["X"].values, Data["Y"].values, Data["Z"].values

K, R, T, M = Pinhole6292(u, v, X, Y, Z)
print(f"{K=}")
print(f"{R=}")
print(f"{T=}")
print(f"{M=}")

In [ ]:
M - K @ np.concatenate([R, T], axis=1)

In [ ]:
r0 = M @ np.array([0, 0, 0, 1])
r1 = M @ np.array([0, 0, 1, 1])

d = r1[:2]/r1[-1] - r0[:2]/r0[-1]

# d /= np.linalg.norm(d)
d


In [ ]:
d.dot(d2)

In [9]:
r0 = M @ np.array([0, 0, 0, 1])
r1 = M @ np.array([0, 0, 1, 1])

d2 = r1[:2]/r1[-1] - r0[:2]/r0[-1]


d2 /= np.linalg.norm(d2)


In [ ]:
Rz = np.array([ 0.58886723, -0.15026282,  0.79413882])

np.arccos(0.79413882)

In [ ]:
R.T @ R

In [ ]:
xyz_homogenous = np.ones((4, len(X)))
xyz_homogenous[0, :] = X
xyz_homogenous[1, :] = Y
xyz_homogenous[2, :] = Z

u_rp, v_rp, h = M @ xyz_homogenous
U_e, V_e = u_rp/h - u, v_rp/h - v
e = (U_e ** 2 + V_e ** 2)

rmse = np.sqrt(1 / len(e) * np.sum(e))
rmse

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.scatter(U_e, V_e)

In [14]:
from CamModels import Pinhole

PC = Pinhole((0, 0), 1)
PC.Fit(u, v, X, Y, Z)

In [ ]:
PC.Map(0, 0, 1) - PC.Map(0, 0, 0)

In [ ]:
PC.R

In [ ]:
R

In [ ]:
np.array([[PC.f, 0, 0],[0, PC.f, 0],[0, 0, 1]]) @ PC.Rt

In [ ]:
PC.RMSE(X, Y, Z, u, v)